In [1]:
import itertools
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.distributions as D
from torch.utils.data import DataLoader, TensorDataset

n = 5
epochs = 3
supervisionEpochs = 5
lr = 0.001
log_interval = 20
trainSize = 100000
penaltyLambda = 10
doublePeakHighMean = 0.9
doublePeakLowMean = 0.1
doublePeakStd = 0.1

d1 = D.normal.Normal(doublePeakLowMean, doublePeakStd)
d2 = D.normal.Normal(doublePeakHighMean, doublePeakStd)
distributionRatio = (d1.cdf(1) + d2.cdf(1) - d1.cdf(0) - d2.cdf(0)) / 2
distributionBase = d1.cdf(0) + d2.cdf(0)


def cdf(x, i=None):
    return (d1.cdf(x) + d2.cdf(x) - distributionBase) / 2 / distributionRatio

# def cdf(x, i=None):
#     if x < 0.1:
#         return 0
#     if x <= 0.2:
#         return 0.5 * (x - 0.1) / 0.1
#     if x < 0.8:
#         return 0.5
#     if x < 0.9:
#         return 0.5 + 0.5 * (x - 0.8) / 0.1
#     return 1


print(distributionBase)

tensor(0.1587)


In [2]:
signals = np.random.randint(2, size=(trainSize, n))
# samples1 = np.random.uniform(low=0.1, high=0.2, size=(trainSize, n))
# samples2 = np.random.uniform(low=0.8, high=0.9, size=(trainSize, n))
samples1 = np.random.normal(
    loc=doublePeakLowMean, scale=doublePeakStd, size=(trainSize, n)
)
for i in range(trainSize):
    for j in range(n):
        while samples1[i, j] < 0 or samples1[i, j] > 1:
            samples1[i, j] = np.random.normal(
                loc=doublePeakLowMean, scale=doublePeakStd
            )
samples2 = np.random.normal(
    loc=doublePeakHighMean, scale=doublePeakStd, size=(trainSize, n)
)
for i in range(trainSize):
    for j in range(n):
        while samples2[i, j] < 0 or samples2[i, j] > 1:
            samples2[i, j] = np.random.normal(
                loc=doublePeakHighMean, scale=doublePeakStd
            )
samplesJoint = signals * samples1 - (signals - 1) * samples2
tp_tensor = torch.tensor(samplesJoint, dtype=torch.float32)
# tp_tensor = torch.tensor(np.random.rand(10000, n), dtype=torch.float32)
tp_dataset = TensorDataset(tp_tensor[: round(trainSize * 0.3)])
tp_dataset_testing = TensorDataset(tp_tensor[round(trainSize * 0.7) :])
tp_dataloader = DataLoader(tp_dataset, batch_size=128, shuffle=True)
tp_dataloader_testing = DataLoader(tp_dataset_testing, batch_size=256, shuffle=False)


# for mapping binary to payments before softmax
model = nn.Sequential(
    nn.Linear(n, 100),
    nn.ReLU(),
    nn.Linear(100, 100),
    nn.ReLU(),
    nn.Linear(100, 100),
    nn.ReLU(),
    nn.Linear(100, n),
)


# optimizer = optim.SGD(model.parameters(), lr=lr)
optimizer = optim.Adam(model.parameters(), lr=lr)


In [3]:
def bitsToPayments(bits):
    if sum(bits).item() == 0:
        return torch.ones(n)
    bits = bits.type(torch.float32)
    negBits = torch.ones(n) - bits
    payments = model(bits)
    payments = payments - 1000 * negBits
    payments = torch.softmax(payments, 0)
    payments = payments + negBits
    return payments


def tpToBits(tp, bits=torch.ones(n).type(torch.uint8)):
    payments = bitsToPayments(bits)
    newBits = (tp >= payments).type(torch.uint8)
    
    if torch.equal(newBits, bits):
        return bits
    else:
        return tpToBits(tp, newBits)#bits-bits#tpToBits(tp, newBits)


def tpToPayments(tp):
    return bitsToPayments(tpToBits(tp))


def tpToTotalDelay(tp):
    return n - sum(tpToBits(tp).type(torch.float32))


In [4]:
print(1_000_000)
print(cdf(0.5))

1000000
tensor(0.5000)


In [5]:
dpPrecision = 100
# howManyPpl left, money left, yes already
dp = np.zeros([n + 1, dpPrecision + 1, n + 1])
decision = np.zeros([n + 1, dpPrecision + 1, n + 1], dtype=np.uint8)
# ppl = 0 left
for yes in range(n + 1):
    for money in range(dpPrecision + 1):
        if money == 0:
            dp[0, 0, yes] = 0
        else:
            dp[0, money, yes] = yes# + 1.0
for ppl in range(1, n + 1):
    for yes in range(n + 1):
        for money in range(dpPrecision + 1):
            minSoFar = 1_000_000
            for offerIndex in range(money + 1):
                offer = offerIndex / dpPrecision
                res = (1 - cdf(offer)) * dp[
                    ppl - 1, money - offerIndex, min(yes + 1, n)
                ] + cdf(offer) * (1 + dp[ppl - 1, money, yes])
                if minSoFar > res:
                    minSoFar = res
                    decision[ppl, money, yes] = offerIndex
            dp[ppl, money, yes] = minSoFar


In [6]:
print(dp[n, dpPrecision, 0])

1.8651759624481201


In [7]:
print(samplesJoint)

[[0.92989751 0.02898375 0.75126922 0.79510375 0.25816531]
 [0.23487141 0.08623337 0.92249064 0.23326167 0.92682494]
 [0.98068152 0.19836839 0.99081174 0.95750388 0.86124423]
 ...
 [0.88304746 0.90298788 0.91299709 0.83780727 0.79305787]
 [0.17102206 0.82916247 0.92689154 0.19402378 0.1579574 ]
 [0.92451844 0.1628815  0.90141356 0.93858076 0.15795591]]


In [8]:
def plan_dp(temp):
    #print(temp)
    remain=dpPrecision
    yes=0;
    ans =0;
    o_list=[];
    remain_list=[];
    for ppl in range(n,0,-1):
        o=decision[ppl, remain, yes]
        #print(o,remain)
        o_list.append(o)
        remain_list.append(remain);
        if(o<temp[n-ppl]):
            remain-=int(o);
            yes+=1;
        elif (remain>0):
            ans+=1;
    if(remain<=0):
        return ans,o_list;
    else:
        return n,o_list;
    

In [9]:
ans_list=[];
for i in range(10000):
    temp=samplesJoint[i]*dpPrecision
    #print(temp)
    ans_list.append(plan_dp(temp)[0]);
    #print("\n",temp)
    #print(plan_dp(temp))
print(sum(ans_list)/len(ans_list))

1.8607


In [10]:
def dpSupervisionRule(tp):
    tp = list(tp.numpy())
    bits = [1 for ii in range(n)]
    payments = [0 for ii in range(n)]
    money = dpPrecision
    yes = 0
    for i in range(n):
        offerIndex = decision[n - i, money, yes]
        offer = offerIndex / dpPrecision
        if tp[i] >= offer:
            money -= offerIndex
            yes += 1
            bits[i] = 1
            payments[i] = offer
        else:
            bits[i] = 0
            payments[i] = 1
    if money > 0:
        bits = [0 for ii in range(n)]
        payments = [1 for ii in range(n)]
    bits = torch.tensor(bits, dtype=torch.uint8)
    payments = torch.tensor(payments, dtype=torch.float32)
    # print()
    # print(tp)
    # print(bits)
    # print(payments)
    # print()
    return (bits, payments)


def dpDelay(tp):
    bits, payments = dpSupervisionRule(tp)
    totalPayment = torch.dot(bits.type(torch.float32), payments).item()
    if totalPayment < 1:
        return n
    else:
        return n - sum(bits).item()


def costSharingSupervisionRule(tp):
    tp = list(tp.numpy())
    for k in range(n, -1, -1):
        if k == 0:
            break
        bits = [1 if tp[ii] >= 1 / k else 0 for ii in range(n)]
        if sum(bits) == k:
            break
    if k == 0:
        payments = [1 for ii in range(n)]
    else:
        payments = [1 / k if bits[ii] == 1 else 1 for ii in range(n)]
    bits = torch.tensor(bits, dtype=torch.uint8)
    payments = torch.tensor(payments, dtype=torch.float32)
    return (bits, payments)


def costSharingDelay(tp):
    return n - sum(costSharingSupervisionRule(tp)[0]).item()

In [11]:
for i in range(n, 0, -1):
            print([1 if ii >= i else 0 for ii in range(n)])
            print(
                    tpToPayments(
                            torch.tensor([0 if ii >= i else 1 for ii in range(n)], dtype=torch.float32)
                    )
                )

[0, 0, 0, 0, 0]
tensor([0.1980, 0.1792, 0.2107, 0.1998, 0.2123], grad_fn=<AddBackward0>)
[0, 0, 0, 0, 1]
tensor([0.2508, 0.2253, 0.2746, 0.2493, 1.0000], grad_fn=<AddBackward0>)
[0, 0, 0, 1, 1]
tensor([0.3378, 0.3118, 0.3504, 1.0000, 1.0000], grad_fn=<AddBackward0>)
[0, 0, 1, 1, 1]
tensor([0.5234, 0.4766, 1.0000, 1.0000, 1.0000], grad_fn=<AddBackward0>)
[0, 1, 1, 1, 1]
tensor([1., 1., 1., 1., 1.], grad_fn=<AddBackward0>)


In [12]:
allBits = [torch.tensor(bits) for bits in itertools.product([0, 1], repeat=n)]
print(allBits)

for batch_idx, (tp_batch,) in enumerate(tp_dataloader_testing):
    penalty = 0
    for bitsMoreOnes in allBits:
        for i in range(n):
            if bitsMoreOnes[i] == 1:
                bitsLessOnes = bitsMoreOnes.clone()
                bitsLessOnes[i] = 0
                penalty = penalty + sum(
                        torch.relu(
                            bitsToPayments(bitsMoreOnes) - bitsToPayments(bitsLessOnes)
                        )
                    )
    loss = penalty * penaltyLambda
    for tp in tp_batch:
            for i in range(n):
                tp1 = tp.clone()
                tp1[i] = 1
                tp0 = tp.clone()
                tp0[i] = 0
                offer = tpToPayments(tp1)[i]
                delay1 = tpToTotalDelay(tp1)
                delay0 = tpToTotalDelay(tp0)
                #loss = loss + (1 - cdf(offer)) * delay1 + cdf(offer) * delay0
                loss = loss + (1 - cdf(offer)) * delay1 + cdf(offer) * delay0
    print()
    print(tp)
    tp1 = tp.clone()
    tp1[0] = 1
    tp0 = tp.clone()
    tp0[0] = 0
    offer = tpToPayments(tp1)[0]
    print(offer)
    print(tpToPayments(tp1))
    print(delay1)
    print(delay0)
    break;
#print(loss)
#print(penalty)

[tensor([0, 0, 0, 0, 0]), tensor([0, 0, 0, 0, 1]), tensor([0, 0, 0, 1, 0]), tensor([0, 0, 0, 1, 1]), tensor([0, 0, 1, 0, 0]), tensor([0, 0, 1, 0, 1]), tensor([0, 0, 1, 1, 0]), tensor([0, 0, 1, 1, 1]), tensor([0, 1, 0, 0, 0]), tensor([0, 1, 0, 0, 1]), tensor([0, 1, 0, 1, 0]), tensor([0, 1, 0, 1, 1]), tensor([0, 1, 1, 0, 0]), tensor([0, 1, 1, 0, 1]), tensor([0, 1, 1, 1, 0]), tensor([0, 1, 1, 1, 1]), tensor([1, 0, 0, 0, 0]), tensor([1, 0, 0, 0, 1]), tensor([1, 0, 0, 1, 0]), tensor([1, 0, 0, 1, 1]), tensor([1, 0, 1, 0, 0]), tensor([1, 0, 1, 0, 1]), tensor([1, 0, 1, 1, 0]), tensor([1, 0, 1, 1, 1]), tensor([1, 1, 0, 0, 0]), tensor([1, 1, 0, 0, 1]), tensor([1, 1, 0, 1, 0]), tensor([1, 1, 0, 1, 1]), tensor([1, 1, 1, 0, 0]), tensor([1, 1, 1, 0, 1]), tensor([1, 1, 1, 1, 0]), tensor([1, 1, 1, 1, 1])]

tensor([0.7833, 0.8669, 0.9186, 0.9144, 0.9911])
tensor(0.1980, grad_fn=<SelectBackward>)
tensor([0.1980, 0.1792, 0.2107, 0.1998, 0.2123], grad_fn=<AddBackward0>)
tensor(0.)
tensor(1.)


In [13]:
allBits = [torch.tensor(bits) for bits in itertools.product([0, 1], repeat=n)]

runningLossNN = []
runningLossCS = []
runningLossDP = []


def recordAndReport(name, source, loss, n=100):
    source.append(loss)
    realLength = min(n, len(source))
    avgLoss = sum(source[-n:]) / realLength
    print(f"{name} ({realLength}): {avgLoss}")


def supervisionTrain(epoch, supervisionRule):
    model.train()
    for batch_idx, (tp_batch,) in enumerate(tp_dataloader):
        optimizer.zero_grad()
        loss = 0
        for tp in tp_batch:
            bits, payments = supervisionRule(tp)
            # print()
            # print("supervision")
            # print(tp)
            # print(bits)
            # print()
            # print(payments)
            # print(bitsToPayments(bits))
            # print()
            loss = loss + F.mse_loss(bitsToPayments(bits), payments)

        loss = loss / len(tp_batch)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print(
                "Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
                    epoch,
                    batch_idx * len(tp_batch),
                    len(tp_dataloader.dataset),
                    100.0 * batch_idx / len(tp_dataloader),
                    loss.item(),
                )
            )


def train(epoch):
    model.train()
    for batch_idx, (tp_batch,) in enumerate(tp_dataloader):
        optimizer.zero_grad()
        penalty = 0
        for bitsMoreOnes in allBits:
            for i in range(n):
                if bitsMoreOnes[i] == 1:
                    bitsLessOnes = bitsMoreOnes.clone()
                    bitsLessOnes[i] = 0
                    penalty = penalty + sum(
                        torch.relu(
                            bitsToPayments(bitsMoreOnes) - bitsToPayments(bitsLessOnes)
                        )
                    )
        loss = penalty * penaltyLambda
        # costSharingLoss = 0
        # dpLoss = 0
        for tp in tp_batch:
            # costSharingLoss += costSharingDelay(tp)
            # dpLoss += dpDelay(tp)
            # print()
            # print("---")
            # print(tp)
            # print(costSharingSupervisionRule(tp))
            # print(dpSupervisionRule(tp))
            # print(costSharingDelay(tp), dpDelay(tp))
            # print()
            for i in range(n):
                tp1 = tp.clone()
                tp1[i] = 1
                tp0 = tp.clone()
                tp0[i] = 0
                offer = tpToPayments(tp1)[i]
                delay1 = tpToTotalDelay(tp1)
                delay0 = tpToTotalDelay(tp0)
                loss = loss + (1 - cdf(offer)) * delay1 + cdf(offer) * delay0

        loss = loss / len(tp_batch) / n
        # costSharingLoss /= len(tp_batch)
        # dpLoss /= len(tp_batch)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print(
                "Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
                    epoch,
                    batch_idx * len(tp_batch),
                    len(tp_dataloader.dataset),
                    100.0 * batch_idx / len(tp_dataloader),
                    loss.item(),
                )
            )
            # recordAndReport("NN", runningLossNN, loss.item())
            # recordAndReport("CS", runningLossCS, costSharingLoss)
            # recordAndReport("DP", runningLossDP, dpLoss)
            # print(dp[n, dpPrecision, 0])
            # print(penalty.item())
            # print(distributionRatio)
            # for i in range(n, 0, -1):
            #     print(
            #         tpToPayments(
            #             torch.tensor([1] * i + [0] * (n - i), dtype=torch.float32)
            #         )
            #     )


In [14]:
def test():
    model.eval()
    with torch.no_grad():
        costSharingLoss = 0
        dpLoss = 0
        nnLoss = 0
        lenLoss= 0
        for (tp_batch,) in tp_dataloader_testing:

            for tp in tp_batch:
                costSharingLoss += costSharingDelay(tp)
                dpLoss += dpDelay(tp)
                nnLoss += tpToTotalDelay(tp)
            lenLoss+=len(tp_batch)
        costSharingLoss /= lenLoss
        dpLoss /= lenLoss
        nnLoss /= lenLoss
        print(lenLoss)
        recordAndReport("NN", runningLossNN, nnLoss)
        recordAndReport("CS", runningLossCS, costSharingLoss)
        recordAndReport("DP", runningLossDP, dpLoss)
        print(dp[n, dpPrecision,0])
        for i in range(n, 0, -1):
            print(
                    tpToPayments(
                            torch.tensor([0 if ii >= i else 1 for ii in range(n)], dtype=torch.float32)
                    )
                )

for epoch in range(1, supervisionEpochs + 1):
    print(distributionRatio)
    #supervisionTrain(epoch, costSharingSupervisionRule)
    supervisionTrain(epoch, dpSupervisionRule)
test()
for epoch in range(1, epochs + 1):
    train(epoch)
    test()


tensor(0.8413)
Train Epoch: 1 [0/30000 (0%)]	Loss: 0.051184
Train Epoch: 1 [2560/30000 (9%)]	Loss: 0.003547
Train Epoch: 1 [5120/30000 (17%)]	Loss: 0.003291
Train Epoch: 1 [7680/30000 (26%)]	Loss: 0.001930
Train Epoch: 1 [10240/30000 (34%)]	Loss: 0.001695
Train Epoch: 1 [12800/30000 (43%)]	Loss: 0.000912
Train Epoch: 1 [15360/30000 (51%)]	Loss: 0.000306
Train Epoch: 1 [17920/30000 (60%)]	Loss: 0.000142
Train Epoch: 1 [20480/30000 (68%)]	Loss: 0.000154
Train Epoch: 1 [23040/30000 (77%)]	Loss: 0.000092
Train Epoch: 1 [25600/30000 (85%)]	Loss: 0.000080
Train Epoch: 1 [28160/30000 (94%)]	Loss: 0.000037
tensor(0.8413)
Train Epoch: 2 [0/30000 (0%)]	Loss: 0.000032
Train Epoch: 2 [2560/30000 (9%)]	Loss: 0.000021
Train Epoch: 2 [5120/30000 (17%)]	Loss: 0.000009
Train Epoch: 2 [7680/30000 (26%)]	Loss: 0.000010
Train Epoch: 2 [10240/30000 (34%)]	Loss: 0.000006
Train Epoch: 2 [12800/30000 (43%)]	Loss: 0.000009
Train Epoch: 2 [15360/30000 (51%)]	Loss: 0.000006
Train Epoch: 2 [17920/30000 (60%)]	Los

In [15]:
PATH="pytorchNN=5-Copy2"
torch.save(model.state_dict(), PATH)

In [16]:
#model = torch.load(PATH)
#model.eval()